In [27]:
import pandas as pd
import nltk
from nltk import ngrams
from nltk.corpus import stopwords
import re
from sklearn.feature_extraction.text import TfidfVectorizer

In [28]:
df = pd.read_csv('https://raw.githubusercontent.com/JaredJose/Roblox-Scraper/main/robloxParentsComments.csv?token=GHSAT0AAAAAAB6VRXEZATEH3S22GPVE4F2KY7VXSQQ')

In [29]:
df.head()

,url,author,date,timestamp,score,comment,comment_id,parent_id
0,https://www.reddit.com/r/RobloxParents/comment...,dianndianna,2021-05-06,1.620311e+09,4,I agree. I have been frustrated since download...,gx5jdbv,NaN
1,https://www.reddit.com/r/RobloxParents/comment...,cutenfuzzie,2021-05-06,1.620337e+09,3,I also agree. I am regretting ever letting my ...,gx79ppg,NaN
2,https://i.redd.it/wahei45uohz41.jpg,Deathly_banana,2020-06-11,1.591900e+09,3,Hehehe secy,ftq05wx,NaN
3,https://i.redd.it/wahei45uohz41.jpg,Afimiliarface,2020-07-01,1.593626e+09,2,I want to report this too what’s the game call...,fwlsxns,NaN
4,https://i.redd.it/wahei45uohz41.jpg,StupidSam1,2020-05-29,1.590754e+09,1,"Report it, these games are from around 2014 an...",fs6xz8e,NaN


In [30]:
df['comment']

0     I agree. I have been frustrated since download...
1     I also agree. I am regretting ever letting my ...
2                                           Hehehe secy
3     I want to report this too what’s the game call...
4     Report it, these games are from around 2014 an...
5     I thought they removed those games now... repo...
6                                                  Same
7                              If you know it i know it
8     They did nothing.\nMaybe its cause i vacinate ...
9                                                     ?
10                                                karen
11                  I dont know how he came to the game
12    This was Just a troll to see how this SR would...
13                                   clickbaited games?
14    Oh okay\n\nThere should have been /s at the en...
15                                             Could be
16    I don't get why Roblox is so popular with elem...
17    Having this problem also. Just downloaded 

In [31]:
df['comment'].str.count('game').sum()

22

In [32]:
text = " ".join(df['comment'].dropna().astype(str))
tokens = nltk.word_tokenize(text)
tokens = [re.sub(r'[^\w\s]', '', token) for token in tokens]
stop_words = set(stopwords.words('english'))
filtered_tokens = [token for token in tokens if token.lower() not in stop_words and token.strip() != '']


bi_grams = ngrams(filtered_tokens, 2)

In [33]:
bi_grams_list = list(bi_grams)
bi_grams_list = [tuple(filter(None, bg)) for bg in bi_grams_list]
print(bi_grams_list)

[('agree', 'frustrated'), ('frustrated', 'since'), ('since', 'downloading'), ('downloading', 'Roblox'), ('Roblox', 'son'), ('son', 'almost'), ('almost', '9'), ('9', 'months'), ('months', 'ago'), ('ago', 'would'), ('would', 'like'), ('like', 'able'), ('able', 'block'), ('block', 'games'), ('games', 'block'), ('block', 'content'), ('content', 'countries'), ('countries', 'due'), ('due', 'less'), ('less', 'monitoring'), ('monitoring', 'even'), ('even', 'make'), ('make', 'curated'), ('curated', 'list'), ('list', 'games'), ('games', 'think'), ('think', 'okay'), ('okay', 'games'), ('games', 'shown'), ('shown', 'allowed'), ('allowed', 'turn'), ('turn', 'account'), ('account', 'restrictions'), ('restrictions', 'basically'), ('basically', 'struggles'), ('struggles', 'find'), ('find', 'much'), ('much', 'anything'), ('anything', 'play'), ('play', 'gets'), ('gets', 'upset'), ('upset', 'going'), ('going', 'game'), ('game', 'game'), ('game', 'find'), ('find', 'let'), ('let', 'play'), ('play', 'really

In [34]:
pattern = r'^\s*(who|what|when|where|why|how)\b.*\?$'
questions = df['comment'].str.extract('({})'.format(pattern), flags=re.IGNORECASE)[0].dropna()

In [35]:
print(questions)

Series([], Name: 0, dtype: object)


In [43]:
bi_grams_list1 = [' '.join(bi_gram) for bi_gram in bi_grams_list if '(' not in bi_gram and ')' not in bi_gram]

freq_dist = nltk.FreqDist(bi_grams_list1)
most_common = freq_dist.most_common(1)
print('The most common bi-gram is:', most_common[0][0], 'with a frequency of', most_common[0][1])

The most common bi-gram is: account restrictions with a frequency of 6


In [44]:
vectorizer = TfidfVectorizer(ngram_range=(2, 2))
tfidf = vectorizer.fit_transform(bi_grams_list1)
feature_names = vectorizer.get_feature_names()
scores = tfidf.toarray()[0]
sorted_scores = sorted(zip(feature_names, scores), key=lambda x: x[1], reverse=True)
for i in range(10):
    print(sorted_scores[i])

('agree frustrated', 1.0)
('2014 botted', 0.0)
('8yearold way', 0.0)
('able block', 0.0)
('able play', 0.0)
('account ideal', 0.0)
('account restrictions', 0.0)
('additional games', 0.0)
('ago would', 0.0)
('agree regretting', 0.0)


In [45]:
sums = tfidf.sum(axis = 0)
data1 = []
for col, term in enumerate(feature_names):
    data1.append( (term, sums[0, col] ))
ranking = pd.DataFrame(data1, columns = ['term', 'rank'])
words = (ranking.sort_values('rank', ascending = False))
print ("\n\nWords : \n", words.head(7))



Words : 
                      term  rank
5    account restrictions   6.0
151     parental controls   3.0
79              game game   3.0
235       without account   3.0
142                nt get   2.0
49           curated list   2.0
158          play without   2.0
